In [2]:
import csv
import cupy as np
import glob
import math
import numpy
import os
import random


In [4]:
mempool = np.get_default_memory_pool()

In [5]:
def sigmoid(t):
  return 1/(1+np.exp(-t))

def sigmoid_der(t):
  derivative = t*(1-t)
  array = np.array(derivative, ndmin = 2)
  return array

def calculate_output(weights, input, bias):
  weighted_sum = np.dot(weights,input) + bias
  output = sigmoid(weighted_sum)
  return output 

def get_expected(file):
  path = os.path.normpath(file)
  tmp = path.split(os.sep)
  filename = tmp[-1]

  if "parasitized" in filename:
    out_expected = 0.99
  elif "uninfected" in filename:
    out_expected = 0.01
    
  return out_expected

def calculate_loss(error):
  sum_square_error = np.sum(error)
  mean_square_error = 1 / len(error) * sum_square_error
  return mean_square_error

def calculate_accuracy(true_positive, false_positive, false_negative, true_negative):
  accuracy = (true_positive + true_negative) / (true_positive + false_positive + false_negative + true_negative)
  return accuracy

def calculate_precision(true_positive, false_positive):
  try:
    precision = true_positive / (true_positive + false_positive)
  except ZeroDivisionError:
    precision = -1
  return precision

def calculate_recall(true_positive, false_negative):
  recall = true_positive / (true_positive + false_negative)
  return recall

def load_malaria(file):
  raw_image_data = np.asarray(Image.open(file), dtype = np.float32)
  image_data = raw_image_data.flatten()
  image_data = (image_data / 255) * 0.98 + 0.01
  return image_data

def backwards_propagation_new(out_output, out_expected, out_hidden1, out_hidden2, out_hidden3, out_hidden4, out_hidden5, hidden2_weights, hidden3_weights, hidden4_weights, hidden5_weights, output_weights, input_data):      

  #backwards propagation for the output weights
  der_error_der_out_output = out_output - out_expected
  der_out_output_der_weightedsum_output = sigmoid_der(out_output)
  der_weightedsum_output_der_weights_output = out_hidden5

  delta_weights_output = np.outer(der_error_der_out_output * der_out_output_der_weightedsum_output, der_weightedsum_output_der_weights_output)

  #backwardspropagation for hidden5_weights

  der_error_der_output_hidden5 = np.outer(der_error_der_out_output * der_out_output_der_weightedsum_output, output_weights)
  der_output_hidden5_der_weightedsum_hidden5 = sigmoid_der(out_hidden5)
  der_weightedsum_hidden5_der_weights_hidden = out_hidden4

  delta_weights_hidden5 = np.outer(der_error_der_output_hidden5 * der_output_hidden5_der_weightedsum_hidden5, der_weightedsum_hidden5_der_weights_hidden)

  #backwardspropagation for hidden4_weights
  der_error_der_output_hidden4 = np.dot(der_error_der_output_hidden5 * der_output_hidden5_der_weightedsum_hidden5, hidden5_weights
)
  der_output_hidden4_der_weightedsum_hidden4 = sigmoid_der(out_hidden4)
  der_weightedsum_hidden4_der_weights_hidden = out_hidden3

  delta_weights_hidden4 = np.outer(der_error_der_output_hidden4 * der_output_hidden4_der_weightedsum_hidden4, der_weightedsum_hidden4_der_weights_hidden)

  #backwardspropagation for hidden3_weights
  der_error_der_output_hidden3 = np.dot(der_error_der_output_hidden4 * der_output_hidden4_der_weightedsum_hidden4, hidden4_weights)
  der_output_hidden3_der_weightedsum_hidden3 = sigmoid_der(out_hidden3)
  der_weightedsum_hidden3_der_weights_hidden = out_hidden2

  delta_weights_hidden3 = np.outer(der_error_der_output_hidden3 * der_output_hidden3_der_weightedsum_hidden3, der_weightedsum_hidden3_der_weights_hidden)

  #backwardspropagation for hidden_weights
  der_error_der_output_hidden2 = np.dot(der_error_der_output_hidden3 * der_output_hidden3_der_weightedsum_hidden3, hidden3_weights)
  der_output_hidden2_der_weightedsum_hidden2 = sigmoid_der(out_hidden2)
  der_weightedsum_hidden2_der_weights_hidden = out_hidden1

  delta_weights_hidden2 = np.outer(der_error_der_output_hidden2 * der_output_hidden2_der_weightedsum_hidden2, der_weightedsum_hidden2_der_weights_hidden)

  #backwards propagation for the input weights
  der_error_der_out_hidden1 = np.dot(der_error_der_output_hidden2 * der_output_hidden2_der_weightedsum_hidden2, hidden2_weights
)
  der_out_hidden1_der_weightedsum_hidden1 = sigmoid_der(out_hidden1)
  der_weightedsum_hidden1_der_weights_input = input_data

  delta_weights_hidden1 = np.outer(der_error_der_out_hidden1 * der_out_hidden1_der_weightedsum_hidden1, der_weightedsum_hidden1_der_weights_input)#1000, 39601

  return delta_weights_output, delta_weights_hidden5, delta_weights_hidden4, delta_weights_hidden3, delta_weights_hidden2, delta_weights_hidden1

In [15]:
training_data = "/home/leon/Documents/Studienprojekt/Gesichtserkennung/Gesichtserkennung/data/images/training_palette"
validation_data = "/home/leon/Documents/Studienprojekt/Gesichtserkennung/Gesichtserkennung/data/images/validation_palette"

try:
  del hidden1_weights
  del hidden2_weights
  del hidden3_weights
  del hidden4_weights
  del hidden5_weights
  del output_weights
  del raw_image_data
  del out_output
  del out_hidden1
  del out_hidden2
  del image_data
except NameError:
  print("No Arrays to delete")

mempool.free_all_blocks()
#input = 39601
#hidden1 = 100
#hidden2 = 10
#output = 1

hidden1_weights = np.array(2 * np.random.random((1000,39601)).astype(np.float32) - 1, ndmin = 2)
hidden2_weights = np.array(2 * np.random.random((800,1000)).astype(np.float32) - 1, ndmin = 2)
hidden3_weights = np.array(2 * np.random.random((600,800)).astype(np.float32) - 1, ndmin = 2)
hidden4_weights = np.array(2 * np.random.random((400,600)).astype(np.float32) - 1, ndmin = 2)
hidden5_weights = np.array(2 * np.random.random((200,400)).astype(np.float32) - 1, ndmin = 2)
output_weights = np.array(2 * np.random.random((1,200)).astype(np.float32) - 1, ndmin = 2)
bias = 0
L = 0.01
epochs = 50

true_positive = 0
false_positive = 0
false_negative = 0
true_negative = 0

No Arrays to delete


In [16]:
training_images = list()
for file in glob.glob(training_data + r"/*.png"):
    training_images.append(file)
random.shuffle(training_images)

In [17]:
run = 0
storage = list()


for epoch in range(epochs):
  run += 1

  true_positive = 0
  false_positive = 0
  false_negative = 0
  true_negative = 0

  error = np.empty((0,0), dtype = np.float32)

  for file in training_images:
    image_data = load_malaria(file)

    out_expected = get_expected(file)

    out_hidden1 = calculate_output(hidden1_weights, image_data, bias)
    out_hidden2 = calculate_output(hidden2_weights, out_hidden1, bias)
    out_hidden3 = calculate_output(hidden3_weights, out_hidden2, bias)
    out_hidden4 = calculate_output(hidden4_weights, out_hidden3, bias)
    out_hidden5 = calculate_output(hidden5_weights, out_hidden4, bias)
    out_output = calculate_output(output_weights, out_hidden5, bias)

    error = np.append(error, ((1 / 2) * np.power((out_expected - out_output), 2)))

    if out_expected > 0.5 and out_output >= 0.5:
      true_positive+=1
    elif out_expected < 0.5 and out_output >= 0.5:
      false_positive+=1
    elif out_expected > 0.5 and out_output < 0.5:
      false_negative+=1
    elif out_expected < 0.5 and out_output < 0.5:
      true_negative+=1
    else:
      print(out_expected, out_output)


    delta_weights_output, delta_weights_hidden5, delta_weights_hidden4, delta_weights_hidden3, delta_weights_hidden2, delta_weights_hidden1 = backwards_propagation_new(out_output, out_expected, out_hidden1, out_hidden2, out_hidden3, out_hidden4, out_hidden5, hidden2_weights, hidden3_weights, hidden4_weights, hidden5_weights, output_weights, image_data)
    
    #update weights
    output_weights = output_weights - L * delta_weights_output
    hidden1_weights = hidden1_weights - L * delta_weights_hidden1
    hidden2_weights = hidden2_weights - L * delta_weights_hidden2
    hidden3_weights = hidden3_weights - L * delta_weights_hidden3
    hidden4_weights = hidden4_weights - L * delta_weights_hidden4
    hidden5_weights = hidden5_weights - L * delta_weights_hidden5

  loss = calculate_loss(error)
  accuracy = calculate_accuracy(true_positive, false_positive, false_negative, true_negative)
  precision = calculate_precision(true_positive, false_positive)
  recall = calculate_recall(true_positive, false_negative)
  
  if loss < 0.09:
    L = 0.001
  elif loss < 0.08:
    L = 0.0001
  elif loss < 0.07:
    L = 0.00001

  tmp_list = [run, true_positive, false_positive, false_negative, true_negative, float(loss), accuracy, precision, recall]
  storage.append(tmp_list)
  print(run, true_positive, false_positive, false_negative, true_negative, loss, accuracy, precision, recall)

1 6252 3199 3487 7760 0.10241153 0.6769736206396754 0.6615172997566395 0.6419550261833864
2 6600 2950 3139 8009 0.09344588 0.7058169871485168 0.6910994764397905 0.6776876476024233
3 6873 2916 2866 8043 0.089317545 0.7206493381002995 0.7021146184492798 0.705719273025978
4 7268 2775 2471 8184 0.082529135 0.7465455599574838 0.723688140993727 0.746277851935517
5 7339 2680 2400 8279 0.08037217 0.7545656585177312 0.732508234354726 0.7535681281445734
6 7392 2638 2347 8321 0.07932687 0.7591554739588366 0.7369890329012961 0.759010165314714
7 7420 2604 2319 8355 0.07841906 0.7621509324572422 0.7402234636871509 0.761885203819694
8 7479 2602 2260 8357 0.077551365 0.7650980771088994 0.7418906854478723 0.7679433206694732
9 7491 2575 2248 8384 0.07696383 0.7669823171320901 0.7441883568448242 0.7691754800287504
10 7516 2568 2223 8391 0.076431796 0.7685283602280414 0.7453391511305038 0.7717424786939111
11 7547 2549 2192 8410 0.07591254 0.7709440525654653 0.7475237717908082 0.7749255570387104
12 7569 25

KeyboardInterrupt: 

In [7]:
tmp_list = list()

true_positive = 0
false_positive = 0
false_negative = 0
true_negative = 0
error = np.empty((0,0), dtype = np.float32)

for file in glob.glob(validation_data + r"/*.png"):
  raw_image_data = np.asarray(Image.open(file), dtype = np.float32)
  image_data = raw_image_data.flatten()
  image_data = (image_data / 255) * 0.98 + 0.01

  out_expected = get_expected(file)

  out_hidden1 = calculate_output(hidden1_weights, image_data, bias)
  out_hidden2 = calculate_output(hidden2_weights, out_hidden1, bias)
  out_hidden3 = calculate_output(hidden3_weights, out_hidden2, bias)
  out_hidden4 = calculate_output(hidden4_weights, out_hidden3, bias)
  out_hidden5 = calculate_output(hidden5_weights, out_hidden4, bias)
  out_output = calculate_output(output_weights, out_hidden5, bias)

  error = np.append(error, ((1 / 2) * np.power((out_expected - out_output), 2)))

  if out_expected > 0.5 and out_output > 0.5:
    true_positive+=1
  elif out_expected < 0.5 and out_output > 0.5:
    false_positive+=1
  elif out_expected > 0.5 and out_output < 0.5:
    false_negative+=1
  elif out_expected < 0.5 and out_output < 0.5:
    true_negative+=1
  else:
    print(out_expected, out_output)
  
loss = calculate_loss(error)
accuracy = calculate_accuracy(true_positive, false_positive, false_negative, true_negative)
precision = calculate_precision(true_positive, false_positive)
recall = calculate_recall(true_positive, false_negative)
run = -1

tmp_list = [run, true_positive, false_positive, false_negative, true_negative, float(loss), accuracy, precision, recall]
storage.append(tmp_list)
    
print(true_positive, false_positive, false_negative, true_negative, loss, accuracy, precision, recall)

# with open("/home/leon/Documents/Studienprojekt/Gesichtserkennung/Gesichtserkennung/result.csv", 'w') as results:
#     wr = csv.writer(results)
#     wr.writerows(storage)
# np.save("/home/leon/Documents/Studienprojekt/Gesichtserkennung/Gesichtserkennung/hidden1_weights.npy", hidden1_weights)
# np.save("/home/leon/Documents/Studienprojekt/Gesichtserkennung/Gesichtserkennung/hidden2_weights.npy", hidden2_weights)
# np.save("/home/leon/Documents/Studienprojekt/Gesichtserkennung/Gesichtserkennung/hidden3_weights.npy", hidden3_weights)
# np.save("/home/leon/Documents/Studienprojekt/Gesichtserkennung/Gesichtserkennung/hidden4_weights.npy", hidden4_weights)
# np.save("/home/leon/Documents/Studienprojekt/Gesichtserkennung/Gesichtserkennung/hidden5_weights.npy", hidden5_weights)
# np.save("/home/leon/Documents/Studienprojekt/Gesichtserkennung/Gesichtserkennung/output_weights.npy", output_weights)

1899 668 546 2062 0.084269404 0.7654106280193237 0.7397740553174912 0.7766871165644171
